In [1]:
# Import packages
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import datetime
from datetime import datetime, timedelta
from shapely.geometry import Point
from shapely.geometry import shape
from statsmodels.tsa.arima.model import ARIMA
from keras.models import Sequential
from keras.layers import Dense
import xgboost as xgb
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, MinMaxScaler
import libpysal as ps
from spreg import GM_Lag
from sklearn.linear_model import LassoCV
import statsmodels.api as sm
import os
from zipfile import ZipFile
import random
import networkx as nx
import warnings

In [2]:
# Open the zip file in read-only mode.

DC_boundary = gpd.read_file('Data/Shapefile/Washington_DC_Boundary.geojson')

def update_datetime(row):
    if int(row[11:13]) >= 24:
        return datetime.strptime(row[:11] + str(int(row[11:13])%24) + row[13:], '%Y-%m-%d %H:%M:%S') + timedelta(days = int(row[11:13])//24) # Increment the day
    else:
        return datetime.strptime(row, '%Y-%m-%d %H:%M:%S')

directory = "Data/GTFS/"
file_paths = []
# crawling through directory and subdirectories
for root, directories, files in os.walk(directory):
    for filename in files:
        # join the two strings in order to form the full filepath.
        filepath = os.path.join(root, filename)
        file_paths.append(filepath)

date_list = []
for i in range (len(file_paths)):
    # Extract the date from the file name
    date_str = os.path.splitext(os.path.basename(file_paths[i]))[0].split('_')[1]
    # Convert the date string to datetime object
    date_obj = datetime.strptime(date_str, '%Y%m%d')
    # Format the datetime object as 'YYYY-MM-DD'
    date_list.append(date_obj.strftime('%Y-%m-%d') + ' ')

transit_stops_info = []
for i in range (231, len(file_paths)):
    archive = ZipFile(file_paths[i], "r")
    trips = pd.read_csv(archive.open('trips.txt'))
    stops = pd.read_csv(archive.open('stops.txt'))
    ## add geometry for stops
    stops['geometry'] = stops[['stop_lon', 'stop_lat']].values.tolist()
    stops['geometry'] = stops['geometry'].apply(Point)
    # select stops within DC
    DC_stop_index = []
    for j in range (len(stops)):
        if stops.geometry[j].within(DC_boundary.geometry[0]):
            DC_stop_index.append(j)
    stops = stops.loc[DC_stop_index]
    stops_time = pd.read_csv(archive.open('stop_times.txt'))
    ## merge
    transit_stops_time = stops_time.merge(stops, how = 'left', on = "stop_id")
    transit_stops_time = transit_stops_time.merge(trips, how = 'left', on = "trip_id")
    # select trip id within DC
    transit_stops_time = transit_stops_time.iloc[np.isin(transit_stops_time['stop_id'], stops.stop_id.unique())]
    # delete unnecessary columns
    transit_stops_time = transit_stops_time.drop(columns = {"trip_id", "stop_sequence", "pickup_type", "drop_off_type", "shape_dist_traveled", "route_id", "service_id", "stop_code", "stop_desc", "stop_name", "zone_id", "block_id", "scheduled_trip_id", "trip_headsign", "direction_id", "shape_id"}, axis = 1)
    # add date information for the dataframe
    transit_stops_time.arrival_time = date_list[i] + transit_stops_time.arrival_time
    transit_stops_time.departure_time = date_list[i] + transit_stops_time.departure_time
    transit_stops_info.append(transit_stops_time)
    print(i)

transit_stops_info = pd.concat(transit_stops_info, ignore_index = True)
transit_stops_info["arrival_time"] = transit_stops_info["arrival_time"].apply(update_datetime)
transit_stops_info["arrival_time"] = pd.to_datetime(transit_stops_info["arrival_time"])
transit_stops_info["departure_time"] = transit_stops_info["departure_time"].apply(update_datetime)
transit_stops_info["departure_time"] = pd.to_datetime(transit_stops_info["departure_time"])
print(len(transit_stops_info))
transit_stops_info.head()

231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
135414117


,arrival_time,departure_time,stop_id,stop_lat,stop_lon,geometry
0,2019-10-15 09:06:00,2019-10-15 09:06:00,5635,38.903297,-77.039502,POINT (-77.039502 38.903297)
1,2019-10-15 09:09:00,2019-10-15 09:09:00,7588,38.898327,-77.027777,POINT (-77.027777 38.898327)
2,2019-10-15 09:11:00,2019-10-15 09:11:00,7474,38.898354,-77.021527,POINT (-77.021527 38.898354)
3,2019-10-15 09:13:00,2019-10-15 09:13:00,12874,38.896121,-77.016312,POINT (-77.016312 38.896121)
4,2019-10-15 09:15:00,2019-10-15 09:15:00,12872,38.897770,-77.006402,POINT (-77.006402 38.89777)


In [3]:
transit_stops_info.to_csv("Data/transit_stops_info_part4.csv", index = False)